In [28]:
import pandas as pd
import features
import glob,re,pickle,os,yaml,datetime,pyperclip

# Choix échantillon

In [2]:
debut=datetime.datetime.now()
filePrefix="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/LongitudinalesRnd/"
filePrefix="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/lexique3/"
sampleFiles=glob.glob(filePrefix+"Longitudinal*.csv")

In [3]:
def prefixEchantillon(numero):
    candidats=[]
    for sample in sampleFiles:
#        m=re.match(ur"^.*/(Longitudinal-%s-T\d+-F\d+)%s-paradigmes\.csv"%(numero,casesType),sample)
        m=re.match(ur"^.*/(Longitudinal-Lexique3-%s-T\d+-F\d+)%s-paradigmes\.csv"%(numero,casesType),sample)
        if m:
            candidats.append(m.group(1))
    if len(candidats)==1:
        return candidats[0]
    else:
        print "PB pas de nom unique correspondant",candidats

In [4]:
numeroEchantillon="00"
sampleType="-X"
casesType="-X-Morphomes"
etapeSwim="-Swim2"
samplePrefix=prefixEchantillon(numeroEchantillon)
initialFile=filePrefix+samplePrefix+"-X-paradigmes.csv"
analysisPrefix=filePrefix+samplePrefix+casesType
predictionsFile=analysisPrefix+"-paradigmes%s.csv"%etapeSwim
print predictionsFile
referenceFile="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/MGC-171229-Verbes3.pkl"

/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/lexique3/Longitudinal-Lexique3-00-T150000000-F73539-X-Morphomes-paradigmes-Swim2.csv


# Variante phonologique

In [5]:
phonologicalMap=sampleType.strip("-")
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [6]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [7]:
dierese={"j":"ij", "w":"uw","H":"yH","i":"ij","u":"uw","y":"yH"}

In [8]:
def checkFrench(prononciation):
    glide2voyelle={"j":"i","w":"u","H":"y"}
    if prononciation:
        if "," in prononciation:
            prononciations=prononciation.split(",")
            setPrononciations=set()
            for element in prononciations:
                setPrononciations.add(checkFrench(element))
            result=",".join(list(setPrononciations))
        else:
            result=recoder(prononciation)
            result=result.replace("Jj","J")
            m=re.match(ur"^(.*[^ieèEaOouy926êôâ])([jwH])$",result)
            if m:
                print ("pb avec un glide final", prononciation)
                voyelle=glide2voyelle[m.group(2)]
                result=m.group(1)+voyelle
            m=re.match(ur"(.*[ptkbdgfsSvzZ][rl])([jwH])(.*)",result)
            if m:
                n=re.search(ur"[ptkbdgfsSvzZ][rl](wa|Hi|wê)",result)
                if not n:
                    glide=m.group(2)
                    result=m.group(1)+dierese[glide]+m.group(3)
            m=re.match(ur"(.*)([iuy])([ieEaOouy].*)",result)
            if m:
                glide=m.group(2)
                result=m.group(1)+dierese[glide]+m.group(3)
            result=result.replace("jj","ij")
    else:
        result=prononciation
    return result

# Paradigmes

In [9]:
initialParadigmes=pd.read_csv(initialFile,sep=";",encoding="utf8")
del initialParadigmes[u"Unnamed: 0"]
initialParadigmes=initialParadigmes.dropna(axis=1,how='all')
initialParadigmesColumns=initialParadigmes.columns.tolist()
listeLexemes=initialParadigmes["lexeme"].tolist()
nbLexemes=len(listeLexemes)

In [10]:
initialForms=pd.melt(initialParadigmes[initialParadigmes["lexeme"].isin(listeLexemes)],id_vars=["lexeme"]).dropna()
initialForms["lexeme-case"]=initialForms["lexeme"]+"-"+initialForms["variable"]
initialForms.drop(labels=["lexeme","variable"],axis=1,inplace=True)
initialForms.set_index(["lexeme-case"],inplace=True)
initialFormsIndex=initialForms.index.tolist()

In [11]:
predictedParadigmes=pd.read_csv(predictionsFile,sep=";",encoding="utf8")
del predictedParadigmes[u"Unnamed: 0"]
predictedParadigmes=predictedParadigmes.loc[:,predictedParadigmes.columns.isin(initialParadigmesColumns)].dropna(axis=1,how='all')
if listeLexemes!=predictedParadigmes["lexeme"].tolist():
    print "PB avec la liste des lexèmes prédits"
if set(initialParadigmesColumns)!=set(predictedParadigmes.columns.tolist()):
    print "PB avec la liste des cases prédites"
    print predictedParadigmes.columns.tolist()
    print initialParadigmesColumns

In [12]:
predictedForms=pd.melt(predictedParadigmes[predictedParadigmes["lexeme"].isin(listeLexemes)],id_vars=["lexeme"]).dropna()
predictedForms["lexeme-case"]=predictedForms["lexeme"]+"-"+predictedForms["variable"]
predictedForms.drop(labels=["lexeme","variable"],axis=1,inplace=True)
predictedForms.set_index(["lexeme-case"],inplace=True)
predictedFormsIndex=predictedForms.index.tolist()

In [13]:
# with open(referenceFile,"rb") as input:
#     lexiqueGold=pickle.load(input)

lexiqueGold=pd.read_pickle(referenceFile)

In [16]:
boyeDefectifs=u"abstraire accroire adirer advenir apparoir assavoir".split(" ")
boyeDefectifs+=u"avenir becter béer bienvenir braire bruire chaloir".split(" ")
boyeDefectifs+=u"choir clore comparoir contrefoutre courre déchoir".split(" ")
boyeDefectifs+=u"déclore dépourvoir discontinuer dissoudre".split(" ")
boyeDefectifs+=u"distraire douer échoir éclore".split(" ")
boyeDefectifs+=u"enclore endêver ensuivre ester extraire faillir".split(" ")
boyeDefectifs+=u"férir forclore forfaire foutre frire gésir".split(" ")
boyeDefectifs+=u"huir impartir issir malfaire méfaire messeoir".split(" ")
boyeDefectifs+=u"moufeter moufter mourir oindre ouïr paître parfaire".split(" ")
boyeDefectifs+=u"poindre quérir raire rassir ravoir reclure revaloir".split(" ")
boyeDefectifs+=u"saillir seoir sourdre soustraire stupéfaire".split(" ")
boyeDefectifs+=u"surfaire titre traire urger".split(" ")
print '[u"'+'", u"'.join(boyeDefectifs)+'"]'

[u"abstraire", u"accroire", u"adirer", u"advenir", u"apparoir", u"assavoir", u"avenir", u"becter", u"béer", u"bienvenir", u"braire", u"bruire", u"chaloir", u"choir", u"clore", u"comparoir", u"contrefoutre", u"courre", u"déchoir", u"déclore", u"dépourvoir", u"discontinuer", u"dissoudre", u"distraire", u"douer", u"échoir", u"éclore", u"enclore", u"endêver", u"ensuivre", u"ester", u"extraire", u"faillir", u"férir", u"forclore", u"forfaire", u"foutre", u"frire", u"gésir", u"huir", u"impartir", u"issir", u"malfaire", u"méfaire", u"messeoir", u"moufeter", u"moufter", u"mourir", u"oindre", u"ouïr", u"paître", u"parfaire", u"poindre", u"quérir", u"raire", u"rassir", u"ravoir", u"reclure", u"revaloir", u"saillir", u"seoir", u"sourdre", u"soustraire", u"stupéfaire", u"surfaire", u"titre", u"traire", u"urger"]


In [32]:
defectifsParadigmes=predictedParadigmes[predictedParadigmes["lexeme"].isin(boyeDefectifs)]
initialDefectifs=initialParadigmes[initialParadigmes["lexeme"].isin(boyeDefectifs)]

# Codage phonétique

In [24]:
cSAMPA=u"SZNJêôârEHO692"
cIPA=[u"ʃ",u"ʒ",u"ŋ",u"ɲ",u"ɛ̃",u"ɔ̃",u"ɑ̃",u"ʁ",u"ɛ",u"ɥ",u"ɔ",u"ə",u"œ",u"ø"]

In [25]:
import unicodedata
def listerUnicode(chaine):
    result=[]
    comb=False
    prec=u""
    for char in chaine:
        print char,
        if unicodedata.combining(char):
            result.append(prec+char)
            prec=""
        else:
            result.append(prec)
            prec=char
    result.append(prec)
    return [r for r in result if r!=""]

ipaIn = listerUnicode(cSAMPA)
ipaOut= cIPA
toipa = dict(zip(ipaIn, ipaOut))

def coderIPA(chaine,table=toipa):
    result=chaine
    for k in table:
        result=result.replace(k,table[k])
    return result

S Z N J ê ô â r E H O 6 9 2


# Tabular de conjugaison

In [30]:
etColours=["orange",
           "brown!50",
           "brown",
           "blue!10",
           "yellow!50",
           "blue!25",
           "teal!50",
           "blue!50",
           "cyan!50",
           "lime",
           "pink",
           "magenta!50",
          ]
etCells=[
    ["pi1P","pi2P"]+["ii"+p+n for p in "123" for n in "SP"],
    ["pi3P"],
    ["pi"+p+"S" for p in "123"],
    ["pP"],
    ["pI2S"],
    ["pI1P","pI2P"],
    ["ps"+p+"S" for p in "123"]+["ps3P"],
    ["ps1P","ps2P"],
    ["inf"],
    ["fi"+p+n for p in "123" for n in "SP"]+["pc"+p+n for p in "123" for n in "SP"],
    ["ai"+p+n for p in "123" for n in "SP"]+["is"+p+n for p in "123" for n in "SP"],
    ["pp"+g+n for g in "MF" for n in "SP"],
        ]
tabTemps={
    "pi":u"ind. prs",
    "ii":u"ind. ipf",
    "ai":u"ind. ps",
    "fi":u"ind. fut",
    "ps":u"subj. prs",
    "is":u"subj. ipf",
    "pc":u"cond. prs",
    "pI":u"imper. prs",
    "inf":u"non-fini"
    }
dictEtColours={}
for nET,ET in enumerate(etCells):
    for c in ET:
        dictEtColours[c]=etColours[nET]
#dictEtColours

In [20]:
def makeTabularParadigmeDF(lexeme,lDF,dictColours,title="",coulLim=False, cat="V",dictMorphomes={}):
    row=lDF[lDF["lexeme"]==lexeme.encode("utf8")]
    return makeTabularParadigme(row,dictColours,title="",coulLim=False, cat="V",dictMorphomes={},lexeme=lexeme)

def makeTabularParadigme(row,dictColours,title="",coulLim=False, cat="V",dictMorphomes={},lexeme=""):
    tabular=[]
    def makeValue(case):
#        if len(row[case])>0 and len(row[case].values[0])>0:
        if case in row and all(row[case].notnull()):
            result=coderIPA(row[case].values[0])
        elif dictMorphomes!={} and case in dictMorphomes:
            altCase=dictMorphomes[case][0]
            if len(row[altCase])>0 and len(row[altCase].values[0])>0:
                result=coderIPA(row[altCase].values[0])
            else:
                result="?"
        else:
            result="?"
        return result
    
    def makeLine6(tenseCode):
        line=[tabTemps[tenseCode]]
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            case=tenseCode+person
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
        return r" & ".join(line)+r"\\"

    def makeLine3(tenseCode):
        line=[tabTemps[tenseCode]]
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            if person in ["2S","1P","2P"]:
                case=tenseCode+person
                if case in dictColours:
                    line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
                else:
                    line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
#                line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
            else:
                line.append(r"---")
        return r" & ".join(line)+r"\\"
    
    def makeLineNF():
        line=["non-fini"]
        for case in ["inf","pP","ppMS","ppMP","ppFS","ppFP"]:
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
#            line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
        return r" & ".join(line)+r"\\"

    def makeLineMF(nombre):
        line=[]
        for genre in "mf":
            case=genre+nombre
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
        return r" & ".join(line)+r"\\"
    
    def makeLineCoulLim():
        line=[]
        for numLimite,limite in enumerate(listLimites):
            line.append(r"\cellcolor{%s}%s"%(listLimCoul[numLimite],"$<$"+str(limite)))
        return r"\hline\hline "+r" & ".join(line)+r"\\"
        
    if cat=="V":
        top=[
            r"\begin{center}",
            r"\begin{tabular}{ccccccc}",
            r"\toprule",
            " & ".join([ur"\textsc{%s}"%lexeme]+[p+n for n in ["sg","pl"] for p in "123" ])+r"\\",
            r"\midrule"
            ]
        bottom=[
            r"\bottomrule",
            r"\end{tabular}\\",
            title,
            r"\end{center}",
            r"\bigskip",
            r""
            ]
        tabular.append("\n".join(top))
        for tenseCode in ["pi","ii","fi","pc", "ps","ai", "is"]:
            tabular.append(makeLine6(tenseCode))
        tabular.append(makeLine3("pI"))
        tabular.append(u"\midrule\n")
        tabular.append(ur"& inf. & part. prés. & \multicolumn{4}{c}{part. passé}\\")
        tabular.append(makeLineNF())
    elif cat=="A":
        top=[
            r"\begin{center}",
            r"\begin{tabular}{cc}",
            r"\toprule"
            ]
        bottom=[
            r"\bottomrule",
            r"\end{tabular}\\",
            title,
            r"\end{center}",
#            r"\bigskip",
            r""
            ]
        tabular.append("\n".join(top))
        for number in "sp":
            tabular.append(makeLineMF(number))
    if coulLim:
        tabular.append(makeLineCoulLim())
    tabular.append("\n".join(bottom))
    return "\n".join(tabular)

# Création des tableaux de conjugaison

In [33]:
defectifsTabular=[]
for v in defectifsParadigmes["lexeme"].tolist():
    defectifsTabular.append(makeTabularParadigmeDF(v,initialDefectifs,dictEtColours))
    defectifsTabular.append(makeTabularParadigmeDF(v,defectifsParadigmes,dictEtColours))
pyperclip.copy("\n".join(defectifsTabular))